In [1]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
lf = pl.scan_csv(
    "../dataset/data_cleaned.csv",
    infer_schema_length=0,
    low_memory=True
)


In [3]:
stats = (
    lf.select([
        pl.len().alias("rows"),
        pl.col("question").str.len_chars().mean().alias("avg_chars"),
        pl.col("question").str.len_chars().max().alias("max_chars"),
    ])
    .collect(streaming=True)
)


C:\Users\hp\AppData\Local\Temp\ipykernel_15060\1139131690.py:2: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  lf.select([


In [4]:
stats

rows,avg_chars,max_chars
u32,f64,u32
193509,45.836018,201


In [5]:
top_words = (
    lf.select(pl.col("question").str.split(" "))
      .explode("question")
      .filter(
          (pl.col("question") != "") &
          (pl.col("question").str.len_chars() > 5)
      )
      .group_by("question")
      .len()
      .sort("len", descending=True)
      .head(50)
      .collect(streaming=True)
)


C:\Users\hp\AppData\Local\Temp\ipykernel_15060\1617914108.py:2: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  lf.select(pl.col("question").str.split(" "))


In [6]:
top_words.head(30)[:20]

question,len
str,u32
"""القياده""",17785
"""البلدين""",9352
"""السكان""",8334
"""البطاله""",7632
"""يسكنها""",5198
…,…
"""السودان""",1991
"""الولايات""",1991
"""الاولمبيه""",1941
